In [ ]:
import pymysql
import pymongo
import certifi

# MySQL Connection
mysql_conn = pymysql.connect(
    host="mysql-st124738-ait-fd9f.d.aivencloud.com",
    port=10766,
    user="avnadmin",
    password="******",
    db="miniproject",
    charset="utf8mb4",
    cursorclass=pymysql.cursors.DictCursor,
)




In [ ]:

import certifi
from pymongo import MongoClient


mongo_client = MongoClient("*******", tlsCAFile=certifi.where())
mongo_db = mongo_client["mini_project_final"]  # Use your database name

print("✅ MongoDB Connection Successful!")





✅ MongoDB Connection Successful!


In [52]:
from bson import ObjectId
from datetime import date
from decimal import Decimal

# Global mappings from MySQL IDs to MongoDB ObjectIds
employee_id_map = {}
project_id_map = {}
task_id_map = {}
client_id_map = {}

# ------------------- 1️⃣ Transfer Employees -------------------
def transfer_employees():
    with mysql_conn.cursor() as cursor:
        cursor.execute("""
            SELECT e.Employee_ID, e.FirstName, e.LastName, e.Supervisor_ID, 
                   e.Email, e.Phone, e.Join_date, 
                   k.KPI_ID, k.Total_Actual_Time, k.Total_Estimated_Time, k.Employee_Rating, 
                   k.Total_Tasks_Completed, k.Total_Tasks_Assigned, 
                   r.Role_Name, r.Description AS Role_Description
            FROM Employee e
            LEFT JOIN KPI k ON e.Employee_ID = k.Employee_ID
            LEFT JOIN Role r ON e.Role_ID = r.Role_ID
        """)
        employees = cursor.fetchall()

    employee_docs = []
    for emp in employees:
        emp_object_id = ObjectId()
        employee_id_map[emp["Employee_ID"]] = emp_object_id

        emp_doc = {
            "_id": emp_object_id,
            "mysql_id": emp["Employee_ID"],
            "first_name": emp["FirstName"],
            "last_name": emp["LastName"],
            "supervisor_id": employee_id_map.get(emp["Supervisor_ID"]),
            "email": emp["Email"],
            "phone": emp["Phone"],
            "join_date": str(emp["Join_date"]) if isinstance(emp["Join_date"], date) else None,
            "role": {
                "name": emp["Role_Name"],
                "description": emp["Role_Description"]
            },
            "KPI": {
                "kpi_id": emp["KPI_ID"],
                "total_actual_time": float(emp["Total_Actual_Time"]) if isinstance(emp["Total_Actual_Time"], Decimal) else emp["Total_Actual_Time"],
                "total_estimated_time": float(emp["Total_Estimated_Time"]) if isinstance(emp["Total_Estimated_Time"], Decimal) else emp["Total_Estimated_Time"],
                "employee_rating": float(emp["Employee_Rating"]) if isinstance(emp["Employee_Rating"], Decimal) else emp["Employee_Rating"],
                "tasks_completed": emp["Total_Tasks_Completed"],
                "tasks_assigned": emp["Total_Tasks_Assigned"]
            }
        }
        employee_docs.append(emp_doc)

    if employee_docs:
        mongo_db.employees.insert_many(employee_docs)
    print("✅ Employee Data Transferred with ObjectId")


# ------------------- 2️⃣ Transfer Clients -------------------
def transfer_clients():
    with mysql_conn.cursor() as cursor:
        cursor.execute("""
            SELECT c.Client_ID, c.Client_Name, c.Contact_person, 
                   c.Address, c.Phone, p.Project_ID
            FROM Client c
            LEFT JOIN Project p ON c.Client_ID = p.Client_ID
        """)
        clients = cursor.fetchall()

    client_docs = {}
    for client in clients:
        client_id = client["Client_ID"]
        if client_id not in client_docs:
            client_object_id = ObjectId()
            client_id_map[client_id] = client_object_id

            client_docs[client_id] = {
                "_id": client_object_id,
                "mysql_id": client_id,
                "name": client["Client_Name"],
                "contact_person": client["Contact_person"],
                "address": client["Address"],
                "phone": client["Phone"],
                "projects": []
            }

        if client["Project_ID"] and client["Project_ID"] in project_id_map:
            project_object_id = project_id_map[client["Project_ID"]]
            if project_object_id not in client_docs[client_id]["projects"]:
                client_docs[client_id]["projects"].append(project_object_id)

    if client_docs:
        mongo_db.clients.insert_many(client_docs.values())
    print("✅ Client Data Transferred with Correct Project References")


# ------------------- 3️⃣ Transfer Projects -------------------
def transfer_projects():
    with mysql_conn.cursor() as cursor:
        cursor.execute("""
            SELECT p.Project_ID, p.Project_Name, p.Status, p.PM_ID, 
                   ps.Description, ps.Budget_in_Dollar, ps.Start_Date, ps.End_Date,
                   t.Task_ID
            FROM Project p
            LEFT JOIN Project_Summary ps ON p.Project_ID = ps.Project_ID
            LEFT JOIN Tasks t ON p.Project_ID = t.Project_ID
        """)
        projects = cursor.fetchall()

    project_docs = {}

    for proj in projects:
        project_id = proj["Project_ID"]

        if project_id not in project_docs:
            project_object_id = ObjectId()
            project_id_map[project_id] = project_object_id

            project_docs[project_id] = {
                "_id": project_object_id,
                "mysql_id": project_id,
                "name": proj["Project_Name"],
                "status": proj["Status"],
                "pm_id": employee_id_map.get(proj["PM_ID"]),  # Store PM_ID as reference to Employee
                "summary": {
                    "description": proj["Description"],
                    "budget": float(proj["Budget_in_Dollar"]) if isinstance(proj["Budget_in_Dollar"], Decimal) else proj["Budget_in_Dollar"],
                    "start_date": str(proj["Start_Date"]) if isinstance(proj["Start_Date"], date) else None,
                    "end_date": str(proj["End_Date"]) if isinstance(proj["End_Date"], date) else None
                },
                "tasks_id": []
            }

        if proj["Task_ID"] and proj["Task_ID"] in task_id_map:
            task_object_id = task_id_map[proj["Task_ID"]]
            if task_object_id not in project_docs[project_id]["tasks_id"]:
                project_docs[project_id]["tasks_id"].append(task_object_id)

    if project_docs:
        mongo_db.projects.insert_many(project_docs.values())
    print("✅ Project Data Transferred to MongoDB (Includes PM_ID and Correct Task References)")


# ------------------- 4️⃣ Transfer Project Teams -------------------
def transfer_project_team():
    with mysql_conn.cursor() as cursor:
        cursor.execute("""
            SELECT pt.Project_ID, pt.Employee_ID, pt.Assigned_Date, pt.End_Date
            FROM Project_Team pt
        """)
        project_teams = cursor.fetchall()

    project_team_docs = {}

    for team in project_teams:
        project_object_id = project_id_map.get(team["Project_ID"])
        if not project_object_id:
            continue  # Skip if Project ID is not found

        if project_object_id not in project_team_docs:
            project_team_docs[project_object_id] = {
                "_id": ObjectId(),
                "project_id": project_object_id,
                "team_members": [],
                "assigned_date": str(team["Assigned_Date"]) if isinstance(team["Assigned_Date"], date) else None,
                "end_date": str(team["End_Date"]) if isinstance(team["End_Date"], date) else None
            }

        if team["Employee_ID"] in employee_id_map:
            employee_object_id = employee_id_map[team["Employee_ID"]]
            if employee_object_id not in project_team_docs[project_object_id]["team_members"]:
                project_team_docs[project_object_id]["team_members"].append(employee_object_id)

    if project_team_docs:
        mongo_db.project_teams.insert_many(project_team_docs.values())
    print("✅ Project Team Data Transferred (PM_ID Removed)")


# ------------------- 5️⃣ Transfer Tasks -------------------
def transfer_tasks():
    with mysql_conn.cursor() as cursor:
        cursor.execute("""
            SELECT t.Task_ID, t.Task_Name, t.Severity, t.Status, t.Estimated_Time,
                   ta.Assignment_ID, ta.Employee_ID, ta.Assignment_Date, ta.Completion_percentage, ta.Approval_status,
                   f.Feedback_ID, f.Feedback_Date, f.Feedback_Comments,
                   tl.Log_ID, tl.Actual_Time, tl.Log_date
            FROM Tasks t
            LEFT JOIN Task_Assignment ta ON t.Task_ID = ta.Task_ID
            LEFT JOIN Feedback f ON ta.Assignment_ID = f.Assignment_ID
            LEFT JOIN Time_Log tl ON ta.Assignment_ID = tl.Assignment_ID
        """)
        tasks = cursor.fetchall()

    task_docs = {}

    for task in tasks:
        task_id = task["Task_ID"]
        
        # Create Task Document if not already created
        if task_id not in task_docs:
            task_object_id = ObjectId()
            task_id_map[task_id] = task_object_id  # Store mapping
            task_docs[task_id] = {
                "_id": task_object_id,
                "mysql_id": task_id,
                "name": task["Task_Name"],
                "severity": task["Severity"],
                "status": task["Status"],
                "estimated_time": float(task["Estimated_Time"]) if isinstance(task["Estimated_Time"], Decimal) else task["Estimated_Time"],
                "assignments": {}  # Dictionary to avoid duplicate assignments
            }

        # Process Task Assignment
        if task["Assignment_ID"]:
            assignment_id = task["Assignment_ID"]

            if assignment_id not in task_docs[task_id]["assignments"]:
                task_docs[task_id]["assignments"][assignment_id] = {
                    "_id": ObjectId(),
                    "mysql_id": assignment_id,
                    "employee_id": employee_id_map.get(task["Employee_ID"]),
                    "assignment_date": str(task["Assignment_Date"]) if isinstance(task["Assignment_Date"], date) else None,
                    "completion_percentage": float(task["Completion_percentage"]) if isinstance(task["Completion_percentage"], Decimal) else task["Completion_percentage"],
                    "approval_status": task["Approval_status"],
                    "feedback": [],  # Multiple feedback entries
                    "time_logs": []  # Multiple time logs
                }

            # Add Feedback (if exists)
            if task["Feedback_ID"]:
                feedback_entry = {
                    "_id": ObjectId(),
                    "mysql_id": task["Feedback_ID"],
                    "date": str(task["Feedback_Date"]) if isinstance(task["Feedback_Date"], date) else None,
                    "comments": task["Feedback_Comments"]
                }
                task_docs[task_id]["assignments"][assignment_id]["feedback"].append(feedback_entry)

            # Add Time Log (if exists)
            if task["Log_ID"]:
                time_log_entry = {
                    "_id": ObjectId(),
                    "mysql_id": task["Log_ID"],
                    "actual_time": float(task["Actual_Time"]) if isinstance(task["Actual_Time"], Decimal) else task["Actual_Time"],
                    "log_date": str(task["Log_date"]) if isinstance(task["Log_date"], date) else None
                }
                task_docs[task_id]["assignments"][assignment_id]["time_logs"].append(time_log_entry)

    # Convert Assignments Dictionary to List for each Task
    for task in task_docs.values():
        task["assignments"] = list(task["assignments"].values())

    # Insert into MongoDB
    if task_docs:
        mongo_db.tasks.insert_many(list(task_docs.values()))
        print("✅ Task Data Transferred to MongoDB (Assignments, Feedback, and Time Logs Embedded)")
    else:
        print("⚠️ No valid Task Data found!")



# ------------------- 🔥 Run All Functions in Order -------------------
transfer_employees()
transfer_tasks()
transfer_projects()
transfer_project_team()
transfer_clients()


✅ Employee Data Transferred with ObjectId
✅ Task Data Transferred to MongoDB (Assignments, Feedback, and Time Logs Embedded)
✅ Project Data Transferred to MongoDB (Includes PM_ID and Correct Task References)
✅ Project Team Data Transferred (PM_ID Removed)
✅ Client Data Transferred with Correct Project References


In [53]:
from pymongo import MongoClient
from datetime import datetime, timedelta
import random


# Generate random datetime
def random_dates():
    created_at = datetime(2020, 1, 1) + timedelta(days=random.randint(0, 1460))  # Between 2020-01-01 and 2024-01-01
    updated_at = created_at + timedelta(days=random.randint(1, 365))  # Updated within a year after created_at
    if updated_at > datetime.today():
        updated_at = datetime.today()  # Ensure updated_at is not in the future
    return created_at, updated_at

# Update collections with `created_at` and `updated_at`
def add_timestamps(collection_name):
    collection = mongo_db[collection_name]
    
    for doc in collection.find({}):  # Fetch all documents
        created_at, updated_at = random_dates()

        collection.update_one(
            {"_id": doc["_id"]},
            {"$set": {
                "created_at": created_at.strftime("%Y-%m-%d %H:%M:%S"),
                "updated_at": updated_at.strftime("%Y-%m-%d %H:%M:%S")
            }}
        )

    print(f"✅ Updated `{collection_name}` with created_at and updated_at")

# List of collections to update
collections = ["employees", "clients", "projects", "tasks", "project_teams"]

# Run for all collections
for collection in collections:
    add_timestamps(collection)


✅ Updated `employees` with created_at and updated_at
✅ Updated `clients` with created_at and updated_at
✅ Updated `projects` with created_at and updated_at
✅ Updated `tasks` with created_at and updated_at
✅ Updated `project_teams` with created_at and updated_at


In [58]:
def update_due_dates():
    with mysql_conn.cursor() as cursor:
        cursor.execute("""
            SELECT Assignment_ID, Due_Date 
            FROM Task_Assignment
            WHERE Due_Date IS NOT NULL
        """)
        assignments = cursor.fetchall()

    updated_count = 0

    for assignment in assignments:
        assignment_id = assignment["Assignment_ID"]
        due_date = str(assignment["Due_Date"]) if isinstance(assignment["Due_Date"], date) else None

        # Update the corresponding assignment inside tasks collection
        result = mongo_db.tasks.update_one(
            { "assignments.mysql_id": assignment_id },  # Find the assignment by MySQL ID
            { "$set": { "assignments.$.due_date": due_date } }  # Update Due Date
        )

        if result.modified_count > 0:
            updated_count += 1

    print(f"✅ Updated Due_Date for {updated_count} assignments in MongoDB.")

# Run the update function
update_due_dates()


✅ Updated Due_Date for 1000 assignments in MongoDB.


In [5]:
from pymongo import MongoClient
from datetime import datetime
from bson import ObjectId


# Function to check and update date fields in a collection
def update_date_fields(collection_name, date_fields):
    collection = mongo_db[collection_name]
    
    cursor = collection.find({})

    updated_count = 0
    for doc in cursor:
        update_needed = False
        update_fields = {}

        for field in date_fields:
            value = doc.get(field)

            if value and isinstance(value, str):  # ✅ Check if it's stored as a string
                try:
                    update_fields[field] = datetime.strptime(value, "%Y-%m-%d")  # ✅ Convert to Date format
                    update_needed = True
                except ValueError:
                    pass  # Skip invalid date values

        if update_needed:
            collection.update_one({"_id": doc["_id"]}, {"$set": update_fields})
            updated_count += 1

    print(f"✅ Updated {updated_count} documents in {collection_name}")

# Function to check and update date fields inside embedded sub-documents
def update_embedded_date_fields(collection_name, array_field, date_fields):
    collection = mongo_db[collection_name]

    cursor = collection.find({})

    updated_count = 0
    for doc in cursor:
        update_needed = False
        updated_array = []

        array_data = doc.get(array_field)

        if isinstance(array_data, list):  # ✅ Ensure it's a list (not a string)
            for item in array_data:
                if not isinstance(item, dict):  # ✅ Skip if it's not a dictionary
                    continue

                updated_item = item.copy()  # ✅ Copy data safely
                for field in date_fields:
                    value = item.get(field)

                    if value and isinstance(value, str):  # ✅ Check if it's stored as a string
                        try:
                            updated_item[field] = datetime.strptime(value, "%Y-%m-%d")  # ✅ Convert to Date
                            update_needed = True
                        except ValueError:
                            pass  # Skip invalid date values

                updated_array.append(updated_item)

            if update_needed:
                collection.update_one({"_id": doc["_id"]}, {"$set": {array_field: updated_array}})
                updated_count += 1

        elif isinstance(array_data, dict):  # ✅ Handle embedded dictionary cases (e.g., `summary`)
            updated_item = array_data.copy()  # ✅ Copy dictionary
            for field in date_fields:
                value = array_data.get(field)

                if value and isinstance(value, str):  # ✅ Convert string dates
                    try:
                        updated_item[field] = datetime.strptime(value, "%Y-%m-%d")
                        update_needed = True
                    except ValueError:
                        pass

            if update_needed:
                collection.update_one({"_id": doc["_id"]}, {"$set": {array_field: updated_item}})
                updated_count += 1

    print(f"✅ Updated {updated_count} documents in {collection_name}.{array_field}")

# ✅ Update date fields for main collections
update_date_fields("employees", ["created_at", "updated_at", "join_date"])
update_date_fields("clients", ["created_at", "updated_at"])
update_date_fields("projects", ["created_at", "updated_at"])
update_date_fields("project_teams", ["assigned_date", "end_date", "created_at", "updated_at"])
update_date_fields("tasks", ["created_at", "updated_at"])

# ✅ Update date fields for embedded sub-documents
update_embedded_date_fields("projects", "summary", ["start_date", "end_date"])  # ✅ Fix for `summary`
update_embedded_date_fields("tasks", "assignments", ["assignment_date"])
update_embedded_date_fields("tasks", "assignments.time_logs", ["log_date"])

print("🎉 All date fields checked and updated successfully!")


✅ Updated 0 documents in employees
✅ Updated 0 documents in clients
✅ Updated 0 documents in projects
✅ Updated 0 documents in project_teams
✅ Updated 0 documents in tasks
✅ Updated 150 documents in projects.summary
✅ Updated 1000 documents in tasks.assignments
✅ Updated 0 documents in tasks.assignments.time_logs
🎉 All date fields checked and updated successfully!


In [7]:
from pymongo import UpdateOne

# Find all documents where due_date is stored as a string
docs = tasks_collection.find({"assignments.due_date": {"$type": "string"}})

bulk_updates = []

for doc in docs:
    updated_assignments = []
    for assignment in doc["assignments"]:
        if isinstance(assignment.get("due_date"), str):
            try:
                # Convert string to datetime
                assignment["due_date"] = datetime.strptime(assignment["due_date"], "%Y-%m-%d")
            except ValueError:
                continue  # Skip if format is invalid
        updated_assignments.append(assignment)
    
    # Add update operation
    bulk_updates.append(
        UpdateOne(
            {"_id": doc["_id"]},
            {"$set": {"assignments": updated_assignments}}
        )
    )

# Execute bulk update
if bulk_updates:
    tasks_collection.bulk_write(bulk_updates)
    print(f"✅ Updated {len(bulk_updates)} documents!")
else:
    print("⚠️ No documents required updates.")


✅ Updated 1000 documents!


In [10]:
def update_date_fields(collection_name, date_fields):
    """
    Updates specified date fields in a given MongoDB collection.
    Converts string date values to MongoDB datetime format.
    """
    collection = mongo_db[collection_name]
    bulk_updates = []
    
    # Find documents where date fields are stored as strings
    query = {"$or": [{field: {"$type": "string"}} for field in date_fields]}
    docs = collection.find(query)

    for doc in docs:
        update_fields = {}
        
        for field in date_fields:
            if isinstance(doc.get(field), str):  # Check if it's a string
                try:
                    update_fields[field] = datetime.strptime(doc[field], "%Y-%m-%d %H:%M:%S")
                except ValueError:
                    try:
                        update_fields[field] = datetime.strptime(doc[field], "%Y-%m-%d")
                    except ValueError:
                        continue  # Skip invalid formats

        if update_fields:
            bulk_updates.append(UpdateOne({"_id": doc["_id"]}, {"$set": update_fields}))

    # Execute bulk update
    if bulk_updates:
        collection.bulk_write(bulk_updates)
        print(f"✅ Updated {len(bulk_updates)} documents in {collection_name}")
    else:
        print(f"⚠️ No updates required for {collection_name}")

# Employees Table
update_date_fields("employees", ["created_at", "updated_at", "join_date"])

# Clients Table
update_date_fields("clients", ["created_at", "updated_at"])

# Projects Table
update_date_fields("projects", ["created_at", "updated_at"])

# Project Teams Table
update_date_fields("project_teams", ["assigned_date", "end_date", "created_at", "updated_at"])

# Tasks Table
update_date_fields("tasks", ["created_at", "updated_at"])



✅ Updated 305 documents in employees
✅ Updated 31 documents in clients
✅ Updated 150 documents in projects
✅ Updated 147 documents in project_teams
✅ Updated 1001 documents in tasks


In [12]:
def update_embedded_date_fields(collection_name, array_field, date_fields):
    """
    Updates date fields within embedded arrays inside MongoDB documents.
    """
    collection = mongo_db[collection_name]
    bulk_updates = []

    docs = collection.find({f"{array_field}.{date_fields[0]}": {"$type": "string"}})

    for doc in docs:
        updated_array = []
        
        for item in doc.get(array_field, []):
            updated_item = item.copy()  # Copy original data
            for field in date_fields:
                if isinstance(item.get(field), str):
                    try:
                        updated_item[field] = datetime.strptime(item[field], "%Y-%m-%d %H:%M:%S")
                    except ValueError:
                        try:
                            updated_item[field] = datetime.strptime(item[field], "%Y-%m-%d")
                        except ValueError:
                            continue  # Skip invalid formats
            updated_array.append(updated_item)

        bulk_updates.append(UpdateOne({"_id": doc["_id"]}, {"$set": {array_field: updated_array}}))

    if bulk_updates:
        collection.bulk_write(bulk_updates)
        print(f"✅ Updated {len(bulk_updates)} documents in {collection_name}.{array_field}")
    else:
        print(f"⚠️ No updates required for {collection_name}.{array_field}")

# Summary in Projects
update_embedded_date_fields("projects", "summary", ["start_date", "end_date"])

# Assignments in Tasks
update_embedded_date_fields("tasks", "assignments", ["assignment_date"])

# Time Logs in Tasks (inside assignments)
update_embedded_date_fields("tasks", "assignments.time_logs", ["log_date"])

# Check if the changes are applied
for collection_name in ["employees", "clients", "projects", "project_teams", "tasks"]:
    print(f"\nChecking {collection_name}...")
    updated_docs = mongo_db[collection_name].find({}, { "_id": 1, "created_at": 1, "updated_at": 1 }).limit(5)
    for doc in updated_docs:
        print(doc)

⚠️ No updates required for projects.summary
⚠️ No updates required for tasks.assignments


BulkWriteError: batch op errors occurred, full error: {'writeErrors': [{'index': 0, 'code': 28, 'errmsg': 'Cannot create field \'time_logs\' in element {assignments: [ { _id: ObjectId(\'67bf654dac60760b8da43e1c\'), mysql_id: 1, employee_id: ObjectId(\'67bf6549ac60760b8da43d24\'), assignment_date: new Date(1741392000000), completion_percentage: 100.0, approval_status: "Pending", feedback: [ { _id: ObjectId(\'67bf654dac60760b8da43e1d\'), mysql_id: 275, date: "2024-12-31", comments: "Reposition Right Foot Bursa and Ligament, Open Approach" } ], time_logs: [ { _id: ObjectId(\'67bf654dac60760b8da43e1e\'), mysql_id: 1, actual_time: 241.0, log_date: "2024-09-25" } ], due_date: new Date(1743379200000) } ]}', 'op': {'q': {'_id': ObjectId('67bf654dac60760b8da43e1b')}, 'u': {'$set': {'assignments.time_logs': []}}, 'multi': False, 'upsert': False}}], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 0, 'nMatched': 0, 'nModified': 0, 'nRemoved': 0, 'upserted': []}

In [14]:
from pymongo import UpdateOne
from datetime import datetime

def update_time_log_dates():
    """
    Updates the `log_date` field inside `time_logs` (which is inside `assignments`)
    in the `tasks` collection, ensuring it's stored in `ISODate` format.
    """

    collection = mongo_db["tasks"]
    bulk_updates = []

    # Find all documents where `log_date` is stored as a string
    docs = collection.find({"assignments.time_logs.log_date": {"$type": "string"}})

    for doc in docs:
        updated_assignments = []

        for assignment in doc.get("assignments", []):
            updated_time_logs = []

            for time_log in assignment.get("time_logs", []):
                if isinstance(time_log.get("log_date"), str):  # Convert only if it's a string
                    try:
                        time_log["log_date"] = datetime.strptime(time_log["log_date"], "%Y-%m-%d")
                    except ValueError:
                        time_log["log_date"] = None  # Handle invalid dates

                updated_time_logs.append(time_log)

            # Replace the old time_logs with updated ones
            assignment["time_logs"] = updated_time_logs
            updated_assignments.append(assignment)

        # Prepare a bulk update operation
        bulk_updates.append(UpdateOne(
            {"_id": doc["_id"]}, 
            {"$set": {"assignments": updated_assignments}}
        ))

    # Execute Bulk Updates
    if bulk_updates:
        collection.bulk_write(bulk_updates)
        print(f"✅ Updated {len(bulk_updates)} documents in `tasks.assignments.time_logs.log_date`")
    else:
        print("⚠️ No updates needed for `tasks.assignments.time_logs.log_date`")

# Run the function
update_time_log_dates()


✅ Updated 1000 documents in `tasks.assignments.time_logs.log_date`
